This folder implements two types of style transfer.

The first one is described in demonic_blender.ipynb, and in https://www.cv-foundation.org/openaccess/content_cvpr_2016/papers/Gatys_Image_Style_Transfer_CVPR_2016_paper.pdf.

The second one is taken from https://arxiv.org/pdf/1705.08086.pdf and reimplemented here.

In [1]:
import os

import torch
from tqdm import tqdm

import images as im

import encoder_decoder_factory as edf

torch.set_default_tensor_type('torch.cuda.FloatTensor')
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
all_content_paths = []

content_path = 'C:\\Users\\ConnardMcGregoire\\Documents\\images\\contents'

content_dir = [
    'Eva',
    'bacon',
    'forest',
    'kimi',
]

content_paths = [
    'OS_FHD.jpg',
    'Megumin.jpg',
    'quenelle.jpg',
    'lepen.jpg',
    'joconde.jpg',
    'ori_01.jpg',
    'elden\\elden_ring_ranni.png',
    'elden\\elden_02.jpg',
]

for d in content_dir:
    for f in os.listdir(content_path + '\\' + d):
        all_content_paths.append(content_path + '\\' + d + '\\' + f)

for p in content_paths:
    all_content_paths.append(content_path + '\\' + p)

all_style_paths = [
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\contents\\elden\\elden_ring_ranni.png',
]

style_path = 'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles'

style_dir = [
    'brick',
    'coal',
    'ember',
    'lava',
    'radagon',
    'slime',
    'water',
    'wood',
]

style_paths = [
    'cubism.jpg',
    'Dusthead.jpg',
    'Ice.jpg',
    'In4.jpg',
    'picasso_01.jpg',
    'picasso_02.jpg',
    'picasso_03.jpg',
    'Hokusai.jpg',
    'Jackson.jpg',
    'moss.jpg',
    'tiger.jpg',
    'VanGogh.jpg',
]

for d in style_dir:
    for f in os.listdir(style_path + '\\' + d):
        all_style_paths.append(style_path + '\\' + d + '\\' + f)

for p in style_paths:
    all_style_paths.append(style_path + '\\' + p)

save_path = 'C:\\Users\\ConnardMcGregoire\\Documents\\images\\results\\'

In [3]:
"""
all_style_paths = [
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\VanGogh.jpg',
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\Hokusai.jpg',
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\contents\\elden\\elden_ring_ranni.png',
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\contents\\elden\\elden_02.jpg',
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\water\\in1.jpg',
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\water\\water_02.jpg',
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\water\\water_03.jpg',
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\water\\water_04.jpg',
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\water\\water_07.jpg',
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\lava\\lava_04.jpg',
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\radagon\\radagon_01.jpg',
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\radagon\\radagon_02.png',
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\radagon\\radagon_04.jpg',
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\radagon\\radagon_06.jpg',
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\tiger.jpg',
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\brick\\brick_01.jpg',
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\brick\\brick_02.jpg',
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\brick\\brick_03.jpg',
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\brick\\brick_04.jpg',
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\brick\\brick_05.jpg',


]
"""

"\nall_style_paths = [\n    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\VanGogh.jpg',\n    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\Hokusai.jpg',\n    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\contents\\elden\\elden_ring_ranni.png',\n    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\contents\\elden\\elden_02.jpg',\n    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\water\\in1.jpg',\n    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\water\\water_02.jpg',\n    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\water\\water_03.jpg',\n    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\water\\water_04.jpg',\n    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\water\\water_07.jpg',\n    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\lava\\lava_04.jpg',\n    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\styles\\radagon\\radagon_01.jpg',\n    'C:\\Users\\ConnardMcGregoire\\Documents\\im

In [4]:
all_content_paths = [
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\contents\\manhatan_01.jpg',
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\contents\\manhatan_02.jpg',
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\contents\\manhatan_03.png',
    'C:\\Users\\ConnardMcGregoire\\Documents\\images\\contents\\manhatan_04.png',
]

In [5]:
def style_transfer(model, c_path, s_path, save_path=None, size_factor=1.0):
    content = im.load_not_too_big_image(c_path).to(device)
    style = im.load_not_too_big_image(s_path, max_size=900*900, size_factor=size_factor).to(device)

    output = model(content, style)
    im.save_image(output, save_path)

In [6]:
model = edf.MultiLevelWCT().to(device)

faster_model = edf.FasterMultiLevelWCT_v2(alpha=0.75).to(device)

In [7]:
with torch.no_grad():
    for c in all_content_paths:
        for s in tqdm(all_style_paths):
            style_transfer(faster_model, c, s, save_path + c.split('\\')[-1].split('.')[0] + '_' + s.split('\\')[-1].split('.')[0] + '.jpg',
                           size_factor=.5)
        
        break

 49%|████▉     | 28/57 [02:41<02:50,  5.88s/it]

In [ ]:
print(list(range(0, 0)))

[]
